# Домашнее задание №3. Embedding word2vec fasttext

In [79]:
import pandas as pd
import gensim.downloader as api
from gensim.models import Word2Vec, FastText
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import string
from sklearn.model_selection import train_test_split
import annoy
import numpy as np
import gensim.models
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

## Задание на поиск похожих по эмбеддингам

1. Объединить в одну выборку

2. На основе word2vec/fasttext/русскоязычная модель rusvectors/слоя Embedding реализовать метод поиска ближайших твитов:

-- на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, 

-- метод должен возвращать 5-ть ближайших твитов к этому запросу

3. Проверить насколько хорошо работают подходы

## План решения

[0. Загрузка и просмотр данных](#section_0)

[1. Функции и процедуры подготовки данных](#section_1)

[2. Методs поиска ближайших твитов](#section_2)

[2.1. Word2vec](#section_2.1)

[2.2. Fasttext](#section_2.2)

[2.3. Русскоязычная модель ruwikiruscorpora_upos_cbow_300_10_2021](#section_2.3)

[2.4. Embedding слой](#section_2.4)

[3. Оценки работы моделей](#section_3)

### 0. Загрузка и просмотр данных <a id='section_0'></a>

Загрузим данные и объединим их в один датасет.

In [80]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive(2).csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)

negative = pd.read_csv('negative(2).csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)

df = positive.append(negative)
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [81]:
#разбиваем данные на train и test
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label)

### 1. Функции и процедуры подготовки данных <a id='section_1'></a>

In [82]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [83]:
#1. Функция преобразования твита (лемматизация, удаление стоп-слов и пунктуации)
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [84]:
#пример обработки твита
preprocess_txt(df['text'].values[0])

['firsttimee',
 'школотый',
 'поверь',
 'самый',
 'd',
 'общество',
 'профилировать',
 'предмет',
 'тип']

In [85]:
#2. Процедура подготовки данных к обучению (создание списка токенов для каждого твита)
assert True

# Preprocess for models fitting
tweet_tokens = []
c = 0
for text in x_train:
    tweet_prep = preprocess_txt(text)
    tweet_tokens.append(tweet_prep)
    c += 1
    if c > 100000:
        break

In [86]:
# подготовленные токены твитов (для x_train)
tweet_tokens = [i for i in tweet_tokens if len(i) > 2]
tweet_tokens[0]

['лёша',
 'лазарев',
 '—',
 'чокнутый',
 'парень',
 'd',
 'надеяться',
 'будущее',
 'друг',
 'httptcoa1ngw1zhzk']

In [87]:
#3. Функция поиска 5 ближайших твитов
def get_response(tweet, index, model, index_map):
    tweet_token = preprocess_txt(tweet)
    vector = np.zeros(300)
    norm = 0
    for word in tweet_token:
        if word in model:
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    similar_tweet = index.get_nns_by_vector(vector, 5, include_distances=True)
    
    result = pd.DataFrame({'tweet': [index_map[i]  for i in similar_tweet[0]], 
                           'distance' : similar_tweet[1]})
    return result

### 2. Методы поиска ближайших твитов <a id='section_2'></a>

In [91]:
# Процедура поиска ближайших твитов
def get_model_index(model):
    model_index = annoy.AnnoyIndex(300 ,'angular') # поиск ближайших соседей

    index_map = {}
    counter = 0

    for tweet in x_train:
        n_model = 0
        index_map[counter] = tweet
        tweet_prep = preprocess_txt(tweet)
        
        vector_model = np.zeros(300) #вектор твита

        for word in tweet_prep:
            if word in model:
                vector_model += model[word]
                n_model += 1
        if n_model > 0:
            vector_model = vector_model / n_model #нормализуем вектор твита
       
        model_index.add_item(counter, vector_model) #добавляем элемент для индексации
     
        counter += 1

        if counter > 100000:
            break

    model_index.build(10) #из индексов создаем лес из 10 деревьев
    return model_index, index_map

### 2.1. Word2wec <a id='section_2.1'></a>

In [92]:
modelW2V = Word2Vec(sentences=tweet_tokens, vector_size=300, window=5, min_count=1)

In [93]:
w2v_index, index_map = get_model_index(modelW2V.wv)

In [94]:
print(x_test.values[0])
get_response(x_test.values[0], w2v_index, modelW2V.wv, index_map)

Крапаю введение для диплома. Пишется пока хреново. Ну не умею я длинно писать. :(


,tweet,distance
0,"@DiKazantseva спасибо Диан , просто оч неприят...",0.008853
1,"@_Nothing_Good_ ага, там еще и не такое писали(",0.009642
2,@KattyMorton она всегда заранее пишет варианты...,0.010095
3,"9.12.13 дадада, ты опять скажешь, что я фигню ...",0.010197
4,С питерского жопореза писал на ЛОР с забаненно...,0.010514


### 2.2. Fasttext <a id='section_2.2'></a>

In [95]:
modelFT = FastText(sentences=tweet_tokens, vector_size=300, min_count=1, window=5, workers=8)

In [96]:
ft_index, index_map = get_model_index(modelFT.wv)

In [97]:
print(x_test.values[0])
get_response(x_test.values[0], ft_index, modelFT.wv, index_map)

Крапаю введение для диплома. Пишется пока хреново. Ну не умею я длинно писать. :(


,tweet,distance
0,"Вот не знала не знала, что от спорта может быт...",0.024794
1,"Глупо, когда говорят ""прощают / слабые"" / Я не...",0.030293
2,@innamango ты просто написала типо кто не зага...,0.032466
3,вот я одна сегодня даже и не думала загадывать...,0.032578
4,"@NShtembulska это чтобы тебя не раздражать, за...",0.032684


### 2.3. Русскоязычная модель ruwikiruscorpora_upos_cbow_300_10_2021  <a id='section_2.3'></a>

In [98]:
modelGLoVE = gensim.models.KeyedVectors.load_word2vec_format('./glove/model.bin', binary=True)  

In [99]:
glove_index, index_map = get_model_index(modelGLoVE)

In [100]:
print(x_test.values[0])
get_response(x_test.values[0], glove_index,modelGLoVE, index_map)

Крапаю введение для диплома. Пишется пока хреново. Ну не умею я длинно писать. :(


,tweet,distance
0,Бляяя 9 дней подряд вставать в 6:20..капецц\nс...,1.414214
1,ВСЁ ТАКИ ОТТЯНУТ СИМОНА((\nПУСТЬ ХОТЯ БЫ ИЗ 10...,1.414214
2,"на Гарри Поттере, моего кота зовут Волан-де-мо...",1.414214
3,"@Farewell_Finn аааа!)) я тоже)) особенно тех, ...",1.414214
4,@Dashagagaloo ой бля не сыпь мне соль на рану(...,1.414214


### 2.4. Embedding слой  <a id='section_2.4'></a>

In [544]:
text_dataset = tf.data.Dataset.from_tensor_slices([x for l in tweet_tokens for x in l]) #список всех токенов из x_train
#создание слоя словаря
vectorize_layer = TextVectorization(
    max_tokens=100000,
    output_mode='int',
    output_sequence_length=1)
#создаем словарь
vectorize_layer.adapt(text_dataset)

#создаем модель
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(1,), dtype=tf.string)) #на вход подается строка
model.add(vectorize_layer)  #получаем тензор (1, 100), содержащий для каждого слова индексы из словаря 
model.add(Embedding(100000, 300, input_length=100, mask_zero=True))

In [545]:
print(x_test.values[0])
v1 = model.predict([x_test.values[0]])[0][0]
v2 = model.predict([x_test.values[1]])[0][0]
v1, v1.shape

Отучил поляка от лола, установил ему доту и вот он сидит с ботами играет, учится) красота


(array([ 3.51896025e-02,  1.93531998e-02,  7.84177706e-03,  1.72587372e-02,
         3.50482948e-02,  2.76690386e-02,  8.73645395e-03, -2.88944840e-02,
         3.05211656e-02,  3.56670134e-02, -3.27259786e-02, -2.52324939e-02,
         1.01154931e-02, -4.65058088e-02,  4.42801975e-02, -4.62143496e-03,
         1.76946260e-02,  3.83440368e-02, -1.16466172e-02,  2.22341754e-02,
        -5.50440699e-03, -3.80901247e-03, -3.13960016e-04, -1.70698278e-02,
         1.18994340e-02, -4.18061242e-02,  4.54228409e-02, -3.41743119e-02,
         3.83315198e-02,  1.96713693e-02, -3.11342832e-02,  9.48014110e-03,
        -5.36371022e-04,  2.68412344e-02, -2.91655418e-02,  2.46965401e-02,
         2.82076709e-02,  2.36405060e-03,  2.49383338e-02,  2.59997696e-03,
        -1.83482766e-02, -2.56215222e-02, -2.98489090e-02,  1.64824836e-02,
         2.49980725e-02, -2.36342084e-02, -2.98985839e-02, -4.67239283e-02,
         5.17304987e-03,  3.42441313e-02, -1.65807717e-02, -3.26831266e-03,
         2.9

In [546]:
# Процедура поиска ближайших твитов
emb_index = annoy.AnnoyIndex(300 ,'angular') # поиск ближайших соседей

emb_map = {}
counter = 0

for tweet in x_train:
    emb_map[counter] = tweet
    vector_emb = model.predict([tweet])[0][0]
    emb_index.add_item(counter, vector_emb) #добавляем элемент для индексации
    counter += 1
        
    if counter > 100000:
        break

emb_index.build(10) #из индексов создаем лес из 10 деревьев

True

In [547]:
def get_response_emb(model, emb_index, tweet, n_similar_tweets):
    vector = model.predict([tweet])[0][0]

    similar_tweet = emb_index.get_nns_by_vector(vector, n_similar_tweets, include_distances=True)
    
    result = pd.DataFrame({'tweet': [emb_map[i]  for i in similar_tweet[0]], 
                           'distance' : similar_tweet[1]
                          })
    return result

In [548]:
print(x_test.values[0])
get_response_emb(model, emb_index, x_test.values[0], 5)

Отучил поляка от лола, установил ему доту и вот он сидит с ботами играет, учится) красота


,tweet,distance
0,не знаю какой фильм посмотреть( уже 4ый начала...,0.0
1,"Как же я скучаю по твоему ""канешно""мне етого н...",0.0
2,"О, Билл! И тебе приветики :D http://t.co/n5cjP...",0.0
3,В Череповце сегодня каток на улицах. Вчера рас...,0.0
4,"Пришлось у мамы Ноут забрать, я так и не смог ...",0.0


### 3. Оценки работы моделей <a id='section_3'></a>

**Вывод:** наиболее близкие твиты ищет модель Word2wec. У русскоязычной модели ruwikiruscorpora_upos_cbow_300_10_2021 и Embwdding слоя проблемы с подсчетом дистации (пока решить не удалось).

**P.S.** Если подскажите в чем ошибка кода, буду благодарна :)